In [1]:
import pickle
    
with open('cluster_abundance_fiducial_covariance.pkl', 'rb') as file:
    cluster_count_cov = pickle.load(file)

In [2]:
# import pickle
    
# with open('cluster_abundance_fiducial_covariance_shot_noise_only.pkl', 'rb') as file:
#     cluster_count_cov = pickle.load(file)

In [ ]:
from aemulusnu_massfunction.emulator import *
from aemulusnu_massfunction.fisher_utils import *

In [ ]:
z_bin_edges = [0.2, 0.4, 0.6, 0.8, 1.0]
richness_bin_edges = [20., 30., 45., 60., 300.]

n_z = len(z_bin_edges) - 1
n_r = len(richness_bin_edges) - 1
cov_matrix_2d = np.zeros((n_z * n_r, n_z * n_r))

for i in range(n_z):
    for j in range(n_z):
        for a in range(n_r):
            for b in range(n_r):
                row_index = i * n_r + a
                col_index = j * n_r + b
                cov_matrix_2d[row_index, col_index] = cluster_count_cov[i, j, a, b]


In [ ]:
cov_inv = np.linalg.inv(cov_matrix_2d)

In [ ]:
labels = {
    '10^9 As': r'10^9A_s',
    'ns': r'n_s',
    'H0': r'H_0',
    'w0': r'w_0',
    'ombh2': r'\Omega_b h^2',
    'omch2': r'\Omega_c h^2',
    'nu_mass_ev': r'\sum m_\nu',
}

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 32
plt.rcParams['font.family'] = 'serif'


In [ ]:
import os

dμdθ_step_size = {}
parameter_ranges = {}  # To store the ranges for which files exist

for parameter_changed in tqdm(fiducial_cosmology.keys()):
    dμdθ_step_size[parameter_changed] = {}
    parameter_ranges[parameter_changed] = {'min': None, 'max': None}
    for log10_rel_step_size in np.arange(-5, 0, .01):
        fname_base = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/cluster_abundance_fisher_%s_%.4f' % (
            parameter_changed, log10_rel_step_size)

        # Check if both files exist
        if os.path.exists(fname_base) and os.path.exists(fname_base + '_neg'):
            with open(fname_base, 'rb') as file:
                cluster_abundance_pos = pickle.load(file)
            with open(fname_base + '_neg', 'rb') as file:
                cluster_abundance_neg = pickle.load(file)

            h = np.abs(fiducial_cosmology[parameter_changed]) * 10**log10_rel_step_size
            tmp_theory_deriv = (cluster_abundance_pos - cluster_abundance_neg) / (2 * h)
            flattened_theory_deriv = np.zeros((n_z * n_r))
            for i in range(n_z):
                for a in range(n_r):
                    flattened_theory_deriv[i * n_r + a] = tmp_theory_deriv[i, a]
            dμdθ_step_size[parameter_changed][np.round(log10_rel_step_size,2)] = flattened_theory_deriv

            # Update the ranges
            if parameter_ranges[parameter_changed]['min'] is None or log10_rel_step_size < parameter_ranges[parameter_changed]['min']:
                parameter_ranges[parameter_changed]['min'] = log10_rel_step_size
            if parameter_ranges[parameter_changed]['max'] is None or log10_rel_step_size > parameter_ranges[parameter_changed]['max']:
                parameter_ranges[parameter_changed]['max'] = log10_rel_step_size



In [ ]:
log10_rel_step_sizes = {}

for parameter, ranges in parameter_ranges.items():
    min_range = round(ranges['min'], 3)
    max_range = round(ranges['max'], 3)
    print(f"For parameter {parameter}, files exist in the range {min_range} to {max_range}.")
    log10_rel_step_sizes[parameter] = np.arange(min_range, max_range, .01)


In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.linalg import cho_solve, cho_factor
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 32
plt.rcParams['font.family'] = 'serif'

# Determine the number of subplots
n_parameters = len(fiducial_cosmology.keys())

# Create a figure and an array of subplots with 1 column
fig, axs = plt.subplots(n_parameters, 1, figsize=(12, 2 * n_parameters))#, sharex=True)

# Create a colormap
colormap = cm.rainbow

# Iterate over the parameters and the corresponding axes
for idx, (parameter_changed, ax) in enumerate(zip(fiducial_cosmology.keys(), axs)):
    Fii_inv = []
    fiducial_value = None
    fiducial_step_size = fiducial_log10_rel_step_size[parameter_changed]
    
    for log10_rel_step_size in log10_rel_step_sizes[parameter_changed]:
        fisher_matrix = np.zeros((n_parameters, n_parameters))
        log10_rel_step_size = np.round(log10_rel_step_size, 2)
        for i, θi in enumerate(fiducial_cosmology.keys()):
            for j, θj in enumerate(fiducial_cosmology.keys()):
                dμl_θi = dμdθ_step_size[θi][fiducial_log10_rel_step_size[θi]]
                dμm_θj = dμdθ_step_size[θj][fiducial_log10_rel_step_size[θj]]
                if(θi == parameter_changed): 
                    dμl_θi = dμdθ_step_size[θi][log10_rel_step_size]
                if(θj == parameter_changed): 
                    dμm_θj = dμdθ_step_size[θj][log10_rel_step_size]
                fisher_matrix[i,j] = np.einsum('l,lm,m', dμl_θi, cov_inv, dμm_θj)

        # Compute the Cholesky decomposition
        cholesky_factor = cho_factor(fisher_matrix)

        # Solve for the inverse of the diagonal element
        e_i = np.zeros_like(fisher_matrix[idx])
        e_i[idx] = 1
        inv_diagonal_element = cho_solve(cholesky_factor, e_i)[idx]

        Fii_inv += [inv_diagonal_element]
        if np.abs(log10_rel_step_size - fiducial_step_size) < 1e-5:
            fiducial_value = inv_diagonal_element

            
            
            
            
    # Get a color from the colormap based on the index
    color = colormap(float(idx) / n_parameters)

    X = 10**(log10_rel_step_sizes[parameter_changed])
    X /= 10**fiducial_step_size
    
    ax.plot(X, ((Fii_inv)), color=color)
#     ax.scatter(X, ((Fii_inv)), color=color)

    ax.scatter(1, fiducial_value, color=color, marker='x')
    ax.axhline(fiducial_value, color=color, linestyle='--')
    
    fill_x = np.array([10**(-10), 10**10])
    fill_y1 = [0.9 * (fiducial_value), 0.9 * (fiducial_value)]
    fill_y2 = [1.1 * (fiducial_value), 1.1 * (fiducial_value)]
    ax.fill_between(fill_x, fill_y1, fill_y2,  color=color, alpha=0.1)

    fill_y1 = [0.99 * (fiducial_value), 0.99 * (fiducial_value)]
    fill_y2 = [1.01 * (fiducial_value), 1.01 * (fiducial_value)]
    ax.fill_between(fill_x, fill_y1, fill_y2,  color=color, alpha=0.2)

    ax.set_xscale('log')
#     ax.set_xlim(.01,1.99)
    ax.set_xlim(10**(-.9), 10**(.9))
    ax.set_xticks([10**(-.5), 10**0, 10**(.5)])
    ax.set_xticklabels([r'-1/2',r'0',r'1/2' ])

    ax.set_ylim([0.88 * (fiducial_value), 1.12 * (fiducial_value)])

    l = labels[parameter_changed]
    ax.set_yticks([])
    ax.set_ylabel(r'$%s$' % (l))
#     ax.axvline(10**-2)
axs[0].set_title(r'(Marginalized) ${[F^{-1}]_{\theta\theta}}$')

s = r'Bands indicates ±10\% and ±1\% variation from ${[F^{-1}]_{\theta\theta}}$ at fiducial step size'
axs[0].text(.99, .90,
        s=s,
        transform=axs[0].transAxes,
        horizontalalignment='right',
        verticalalignment='top',
       fontsize=16)

axs[-1].set_xlabel(r'$\log_{10}$(Step Size / Fiducial Step Size)')
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('marginalized_convergencee.pdf')

In [ ]:
# import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.linalg import cho_solve, cho_factor
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 32
plt.rcParams['font.family'] = 'serif'

# Determine the number of subplots
n_parameters = len(fiducial_cosmology.keys())

# Create a figure and an array of subplots with 1 column
fig, axs = plt.subplots(n_parameters, 1, figsize=(12, 2 * n_parameters))#, sharex=True)

# Create a colormap
colormap = cm.rainbow

# Iterate over the parameters and the corresponding axes
for idx, (parameter_changed, ax) in enumerate(zip(fiducial_cosmology.keys(), axs)):
    Fii_inv = []
    fiducial_value = None
    fiducial_step_size = fiducial_log10_rel_step_size[parameter_changed]
    
    for log10_rel_step_size in log10_rel_step_sizes[parameter_changed]:
        fisher_matrix = np.zeros((n_parameters, n_parameters))
        log10_rel_step_size = np.round(log10_rel_step_size, 2)
        for i, θi in enumerate(fiducial_cosmology.keys()):
            for j, θj in enumerate(fiducial_cosmology.keys()):
                dμl_θi = dμdθ_step_size[θi][fiducial_log10_rel_step_size[θi]]
                dμm_θj = dμdθ_step_size[θj][fiducial_log10_rel_step_size[θj]]
                if(θi == parameter_changed): 
                    dμl_θi = dμdθ_step_size[θi][log10_rel_step_size]
                if(θj == parameter_changed): 
                    dμm_θj = dμdθ_step_size[θj][log10_rel_step_size]
                fisher_matrix[i,j] = np.einsum('l,lm,m', dμl_θi, cov_inv, dμm_θj)

        inv_diagonal_element = fisher_matrix[idx][idx]**(-1)

        Fii_inv += [inv_diagonal_element]
        if np.abs(log10_rel_step_size - fiducial_step_size) < 1e-5:
            fiducial_value = inv_diagonal_element

            
            
            
            
    # Get a color from the colormap based on the index
    color = colormap(float(idx) / n_parameters)

    X = 10**(log10_rel_step_sizes[parameter_changed])
    X /= 10**fiducial_step_size
    
    ax.plot(X, ((Fii_inv)), color=color)
#     ax.scatter(X, ((Fii_inv)), color=color)

    ax.scatter(1, fiducial_value, color=color, marker='x')
    ax.axhline(fiducial_value, color=color, linestyle='--')
    
    fill_x = np.array([10**(-10), 10**10])
    fill_y1 = [0.9 * (fiducial_value), 0.9 * (fiducial_value)]
    fill_y2 = [1.1 * (fiducial_value), 1.1 * (fiducial_value)]
    ax.fill_between(fill_x, fill_y1, fill_y2,  color=color, alpha=0.1)

    fill_y1 = [0.99 * (fiducial_value), 0.99 * (fiducial_value)]
    fill_y2 = [1.01 * (fiducial_value), 1.01 * (fiducial_value)]
    ax.fill_between(fill_x, fill_y1, fill_y2,  color=color, alpha=0.2)

    ax.set_xscale('log')
#     ax.set_xlim(.01,1.99)
    ax.set_xlim(10**(-.9), 10**(.9))
    ax.set_xticks([10**(-.5), 10**0, 10**(.5)])
    ax.set_xticklabels([r'-1/2',r'0',r'1/2' ])

    ax.set_ylim([0.88 * (fiducial_value), 1.12 * (fiducial_value)])

    l = labels[parameter_changed]
    ax.set_yticks([])
    ax.set_ylabel(r'$%s$' % (l))
#     ax.axvline(10**-2)
axs[0].set_title(r'(Unmarginalized) ${[F_{\theta\theta}]^{-1}}$')

s = r'Bands indicates ±10\% and ±1\% variation from ${[F_{\theta\theta}]^{-1}}$ at fiducial step size'
axs[0].text(.99, .90,
        s=s,
        transform=axs[0].transAxes,
        horizontalalignment='right',
        verticalalignment='top',
       fontsize=16)

axs[-1].set_xlabel(r'$\log_{10}$(Step Size / Fiducial Step Size)')
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('unmarginalized_convergencee.pdf')